In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"

next_week = 10

#Chose features to drop
feature_drop = ['Movement', 'Team']

#Set max rank hyperparameter
max_rank = 26

#Read in df
df = pd.read_excel(file)
df

,Team,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank,Movement
0,Alabama,1,1,0,0,1,14,1,0,W,44,13,31,1,0
1,Oklahoma,1,2,0,0,1,26,0,1,W,40,35,5,4,-2
2,Clemson,1,3,0,0,1,5,1,0,L,3,10,-7,6,-3
3,Ohio State,1,4,0,0,1,26,1,0,W,45,31,14,3,1
4,Georgia,1,5,0,0,1,3,1,0,W,10,3,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Penn State,10,22,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,SMU,10,23,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,Louisiana-Lafayette,10,24,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,Fresno State,10,25,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop overflow
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
#Encode Result column
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
#Get next week teams list
next_teams = df['Team'].loc[df['Week'] == next_week - 1]
next_rank_act = df['Next Week Rank'].loc[df['Week'] == next_week - 1]

#Get byes
byes_index = []
for index, row in df.iterrows():
    if row['Week'] == next_week - 1 and not(row['Opp. Rank'] >= 0):
        byes_index.append(index % 25)
        
byes_index

[2, 13, 22]

In [6]:
#Drop null rows (teams with byes)
df.dropna(inplace = True)

In [7]:
#Drop irrelevant cols
df = df.drop(columns = feature_drop)

In [8]:
#Convert cols to num
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)

In [9]:
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [10]:
#Set training data to Week < next week
train_df = df.loc[df['Week'] < next_week - 1]
X_train = train_df.drop(columns = ['Next Week Rank'])
y_train = train_df['Next Week Rank']

In [11]:
#Set testing data to Week = next week
test_df = df.loc[df['Week'] == next_week - 1]
X_test = test_df.drop(columns = ['Next Week Rank'])
y_test = test_df['Next Week Rank']

In [12]:
#Multiple Linear Regression
lin_model = LinearRegression().fit(X_train, y_train)
lin_model_output = lin_model.predict(X_test)
lin_model.score(X_test, y_test)

0.9469411626335765

In [13]:
#Add back teams with byes to output
next_rank_pred = lin_model_output.tolist()
for item in byes_index:
    next_rank_pred.insert(item, 'bye')

In [14]:
model_output = lin_model_output.tolist()
model_output

for item in byes_index:
    model_output.insert(item + 1, item + 1)
    
model_output.sort()    

sorted_output = model_output

In [15]:
df2 = pd.DataFrame()
df2['Predicted Rank'] = sorted_output
df2['Predicted Ordinal Rank'] = range(1, 26)

In [16]:
results = pd.DataFrame()
results['Team'] = next_teams
results['Predicted Rank'] = next_rank_pred
results['Actual Rank'] = next_rank_act

results.reset_index(inplace = True, drop = True)

#Predict that teams with byes don't change rank
for index, row in results.iterrows():
    if row['Predicted Rank'] == 'bye':
        results.at[index, 'Predicted Rank'] = index + 1

results['Predicted Rank'] = pd.to_numeric(results['Predicted Rank'], errors = 'coerce')
results['Actual Rank'] = pd.to_numeric(results['Actual Rank'])
results['Previous Rank'] = range(1, 26)

merged_results = results.merge(df2, how = 'left', left_on = 'Predicted Rank', right_on = 'Predicted Rank')

In [17]:
apor_col = []

for index, row in merged_results.iterrows():
    if row['Predicted Rank'] > 25:
        apor_col.append(max_rank)
    
    else:
        apor_col.append(row['Predicted Ordinal Rank'])

In [18]:
merged_results['Adj. Predicted Ordinal Rank'] = apor_col

In [19]:
merged_results = merged_results[['Team', 'Previous Rank', 'Predicted Rank', 'Predicted Ordinal Rank', 'Adj. Predicted Ordinal Rank', 'Actual Rank']]
merged_results

,Team,Previous Rank,Predicted Rank,Predicted Ordinal Rank,Adj. Predicted Ordinal Rank,Actual Rank
0,Georgia,1,1.115134,1,1,1
1,Cincinnati,2,3.396054,5,5,2
2,Alabama,3,3.000000,4,4,3
3,Oklahoma,4,2.287600,2,2,4
4,Ohio State,5,4.035795,6,6,6
5,Michigan,6,11.027533,11,11,9
6,Oregon,7,6.229065,7,7,7
7,Michigan State,8,2.783842,3,3,5
8,Iowa,9,18.921699,16,16,19
9,Ole Miss,10,17.623142,15,15,15


In [20]:
import sys
sys.path.append('../libraries')

from ordinal.ordinal import get_error

In [21]:
get_error(merged_results['Predicted Ordinal Rank'], merged_results['Actual Rank'], mode = 'total')

35

In [22]:
get_error(merged_results['Adj. Predicted Ordinal Rank'], merged_results['Actual Rank'], mode = 'total')

35

In [23]:
merged_results.to_csv(f'week{next_week}_predictions.csv', header = True, index = False)